In [13]:
import cv2
import pandas as pd
import numpy as np

import CNN_Model_1 as CNN_Model

In [16]:
if __name__ == "__main__":

    if True:
        
        for i in range(10):
            
            predict_images(i+1)
     

____________________________________________________________________________________________________________________________________________
Layer (type)                                                   Output Shape                                            Param #              
input (InputLayer)                                             (None, 256, 256, 3)                                     0                    
____________________________________________________________________________________________________________________________________________
conv1a (Conv2D)                                                (None, 256, 256, 32)                                    896                  
____________________________________________________________________________________________________________________________________________
pool1 (MaxPooling2D)                                           (None, 128, 128, 32)                                    0                    
_____________

12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
____________________________________________________________________________________________________________________________________________
Layer (type)                                                   Output Shape                                            Param #              
input (InputLayer)                                             (None, 256, 256, 3)                                     0                    
____________________________________________________________________________________________________________________________________________
conv1a (Conv2D)                                                (None, 256, 256, 32)                                    896                  
____________________________________________________________________________________________________________________________________________
pool1 (MaxPooling2D)                                           (None, 128, 128, 32)                 

12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
____________________________________________________________________________________________________________________________________________
Layer (type)                                                   Output Shape                                            Param #              
input (InputLayer)                                             (None, 256, 256, 3)                                     0                    
____________________________________________________________________________________________________________________________________________
conv1a (Conv2D)                                                (None, 256, 256, 32)                                    896                  
____________________________________________________________________________________________________________________________________________
pool1 (MaxPooling2D)                                           (None, 128, 128, 32)                 

8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
____________________________________________________________________________________________________________________________________________
Layer (type)                                                   Output Shape                                            Param #              
input (InputLayer)                                             (None, 256, 256, 3)                                     0                    
____________________________________________________________________________________________________________________________________________
conv1a (Conv2D)                                                (None, 256, 256, 32)                                    896                  
____________________________________________________________________________________________________________________________________________
pool1 (MaxPooling2D)                                           (None, 128, 128, 32)       

13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
____________________________________________________________________________________________________________________________________________
Layer (type)                                                   Output Shape                                            Param #              
input (InputLayer)                                             (None, 256, 256, 3)                                     0                    
____________________________________________________________________________________________________________________________________________
conv1a (Conv2D)                                                (None, 256, 256, 32)                                    896                  
____________________________________________________________________________________________________________________________________________
pool1 (MaxPooling2D)                                           (None, 128, 128, 32)                       

11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [14]:
def get_train_holdout_files(current_iteration = 1):
    
    print("Get train/holdout files.")
        
    src_dir = "D:/jupyter-notebook/LiverCancer/Data_Description/" + str(current_iteration) +"/"

    #分割训练数据和测试数据  

    holdout_samples = pd.read_csv(src_dir + "holdout.csv")["file_path"].tolist()
    
    return holdout_samples
    

In [15]:
def predict_images(current_iteration = 1):
    
    #读取model
    if current_iteration == 1:
        model_path = "D:/jupyter-notebook/LiverCancer/workdir/model_liver_CNN_1__e09-0.0198.hd5"
    elif current_iteration == 2:
        model_path = "D:/jupyter-notebook/LiverCancer/workdir/model_liver_CNN_2__e10-0.0055.hd5"
    elif current_iteration == 3:
        model_path = "D:/jupyter-notebook/LiverCancer/workdir/model_liver_CNN_3__e10-0.2880.hd5"
    elif current_iteration == 4:
        model_path = "D:/jupyter-notebook/LiverCancer/workdir/model_liver_CNN_4__e09-0.2433.hd5"
    elif current_iteration == 5:
        model_path = "D:/jupyter-notebook/LiverCancer/workdir/model_liver_CNN_5__e07-0.2026.hd5"
    elif current_iteration == 6:
        model_path = "D:/jupyter-notebook/LiverCancer/workdir/model_liver_CNN_6__e09-0.1040.hd5"
    elif current_iteration == 7:
        model_path = "D:/jupyter-notebook/LiverCancer/workdir/model_liver_CNN_7__e07-0.2420.hd5"
    elif current_iteration == 8:
        model_path = "D:/jupyter-notebook/LiverCancer/workdir/model_liver_CNN_8__e09-0.2353.hd5"
    elif current_iteration == 9:
        model_path = "D:/jupyter-notebook/LiverCancer/workdir/model_liver_CNN_9__e10-0.0510.hd5"
    elif current_iteration == 10:
        model_path = "D:/jupyter-notebook/LiverCancer/workdir/model_liver_CNN_10__e09-0.1100.hd5"
        
    csv_target_path = "D:/jupyter-notebook/LiverCancer/Predict_Result/" + str(current_iteration) + "_result.csv"
        
    model = CNN_Model.get_net(load_weight_path=model_path)

    holdout_files = get_train_holdout_files(current_iteration)

    batch_size = 32
    done_count = 0  
    info_list = []
    image_list = []
    out_csv = []

    for index, sample_path in enumerate(holdout_files):
        
        print(index)
        
        if "grade0" in sample_path:
            sample_label = 0
        elif "grade1" in sample_path:
            sample_label = 1
        elif "grade2" in sample_path:
            sample_label = 2
        elif "grade3" in sample_path:
            sample_label = 3
        elif "grade4" in sample_path:
            sample_label = 4

        info_list.append((sample_path,sample_label))

        sample_image = cv2.imread(sample_path)
        sample_image = (sample_image - np.average(sample_image)) / np.std(sample_image)
        sample_image = sample_image.reshape(1, sample_image.shape[0], sample_image.shape[1], 3)
        image_list.append(sample_image)

        ##############################################################################################
        #凑够32个，就开始进行预测
        done_count += 1

        if len(image_list)% batch_size==0 or done_count==len(holdout_files):

            x = np.vstack(image_list)
            p_grade = model.predict({"input":x}, batch_size=batch_size)

            for i in range(len(p_grade)):

                grade_p_0 = str(p_grade[i][0])
                grade_p_1 = str(p_grade[i][1])
                grade_p_2 = str(p_grade[i][2])
                grade_p_3 = str(p_grade[i][3])
                grade_p_4 = str(p_grade[i][4])
                sample_path=info_list[i][0]
                sample_label=info_list[i][1]

                csv_line = [sample_path, grade_p_0, grade_p_1, grade_p_2, grade_p_3, grade_p_4, sample_label]
                out_csv.append(csv_line)

            info_list = []    
            image_list = []

    ###########################################################################################
    #写成csv文件
    df = pd.DataFrame(out_csv, columns=["sample_path", "grade_p_0", "grade_p_1", "grade_p_2", "grade_p_3", "grade_p_4","sample_label"])
    df.to_csv(csv_target_path, index=False)
